In [1]:
from pathlib import Path
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.chains.llm import LLMChain
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate,
)

In [2]:
llm3 = ChatOpenAI(temperature=0, max_tokens=2000, model="gpt-3.5-turbo")
llm3_16 = ChatOpenAI(temperature=0, max_tokens=7000, model="gpt-3.5-turbo-16k")

davinci = OpenAI(temperature=0, max_tokens=2000, model="text-davinci-003")

davinci16k = OpenAI(temperature=0, max_tokens=2000, model="davinci-002")

# llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Map reduce

In [3]:
map_reduce_2015_fname = "map_reduce_summarized_2015.txt"
map_reduce_2023_fname = "map_reduce_summarized_2023.txt"

In [4]:
with open(map_reduce_2015_fname, "r") as f:
    map_reduce_2015 = f.read()

with open(map_reduce_2023_fname, "r") as f:
    map_reduce_2023 = f.read()

## Option 1 - two inputs
### GPT3

In [10]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""Find key differences between the following texts, specified by three 
    backticks highlighting specifically when items exist in 
    one and not the other.
    ```Text 1: {text_2015}```
    ```Text 2: {text_2023}```
    """,
    input_variables=["text_2015", "text_2023"],
)


chat_prompt_with_values = prompt.format_prompt(
    text_2015=map_reduce_2015, text_2023=map_reduce_2023
)

chat_prompt_with_values.to_messages()
response = llm3(chat_prompt_with_values.to_messages()).content

In [12]:
print(response)

The key differences between the two texts are as follows:

1. Text 1 includes the following information that is not present in Text 2:
- Users are responsible for paying for all purchases made through the stores and iTunes may charge their payment method for these purchases.
- Users are not allowed to modify, sell, distribute, or create derivative works based on the Service.
- Users may automatically download and access previously purchased content on multiple devices.
- Users are bound by the usage rules and security framework established by iTunes and the Publishers.
- iTunes may modify the agreement and impose new terms or conditions at any time.
- Users must be at least 13 years old to create an account and are responsible for maintaining the security of their account.
- Users are not allowed to violate or tamper with the security framework or access the stores in any unauthorized way.
- Users are responsible for any materials they submit and must not infringe on the rights of othe

# Davinci

In [94]:
prompt = PromptTemplate(
    template="""Find key differences between the following texts, specified by three 
    backticks
    ```{text_2015}``` and ```{text_2023}```, jhihglighting specifically when items exist in 
    one and not the other, 
    """,
    input_variables=["text_2015", "text_2023"],
)

final_prompt = prompt.format(text_2015=map_reduce_2015, text_2023=map_reduce_2023)
completion = davinci(final_prompt)
print(completion)


1. The first text states that users are responsible for notifying iTunes of any security breaches, while the second text does not mention this. 
2. The first text states that users may be provided with limited access to download album cover art for music stored in their iTunes Library, while the second text does not mention this. 
3. The second text states that users must notify iTunes of any security breaches, while the first text does not mention this. 
4. The second text states that users must provide accurate information when registering and updating their account, while the first text does not mention this. 
5. The second text states that the agreement includes specific terms for the Mac App Store and App Store, including licensing and in-app purchases, while the first text does not mention this.


## Option 2 - single input

In [85]:
delimiter = "#" * 10
combined_text = delimiter + map_reduce_2015 + delimiter + map_reduce_2015 + delimiter

In [86]:
print(f"Your total summary has {llm3.get_num_tokens(combined_text)} tokens")

Your total summary has 1806 tokens


In [87]:
prompt = PromptTemplate(
    template="""The following text contains two summaries, seperated by the following delimiter:
    ##########. Can you find key differences between the two summaries. 
    Text: {text}
    """,
    input_variables=["text"],
)

final_prompt = prompt.format(text=combined_text)
completion = davinci(final_prompt)
print(completion)

# Vector summarizartion

In [48]:
vectors_2015_fname = "vectors_summarized_2015.txt"
vectors_2023_fname = "vectors_summarized_2023.txt"

In [58]:
with open(vectors_2015_fname, "r") as f:
    vector_2015 = f.read()

with open(vectors_2023_fname, "r") as f:
    vector_2023 = f.read()

In [59]:
system_message_prompt.format(text_2015=vector_2015, text_2023=vector_2023)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])
chat_prompt_with_values = chat_prompt.format_prompt(
    text_2015=vector_2015, text_2023=vector_2023
)

chat_prompt_with_values.to_messages()
response = llm3_16(chat_prompt_with_values.to_messages()).content

SystemMessage(content='Find key differences between the following texts, specified by three \n    backticks\n    ```This passage provides information about various features and policies related to purchasing and using products from the Apple Stores. It begins by explaining the convenience of the 1-Click feature, which allows users to make purchases with a single click on their computer or by tapping a button on their Apple-branded devices. When 1-Click is activated, the purchase is immediately downloaded and completed without any further steps.\n\nThe passage also discusses the terms and conditions for Gift Certificates, iTunes Cards and Codes, Content Codes, and Allowances. These items are not redeemable for cash, cannot be returned for a cash refund, and have restrictions on their use. They may only be redeemed in the United Kingdom and are not transferable. Additionally, iTunes is not responsible for lost or stolen gift certificates or codes.\n\nThe passage also includes important s

In [64]:
print(response)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


## Davinci

In [97]:
prompt = PromptTemplate(
    template="""Find key differences between the following texts, specified by three 
    backticks
    ```{text_2015}``` and ```{text_2023}```, jhihglighting specifically when items exist in 
    one and not the other, 
    """,
    input_variables=["text_2015", "text_2023"],
)

final_prompt = prompt.format(text_2015=vector_2015, text_2023=vector_2023)
completion = davinci16k(final_prompt)
print(completion)

 ```This passage discusses the terms and conditions for using the iTunes Service. It states that in order to use the service, users must have compatible devices, internet access, and certain software. High-speed internet access is recommended, especially for film rentals. Users are responsible for meeting these requirements, which may change over time. The passage also emphasizes that the service is separate from any other product or offering, and purchasing other products does not guarantee access to the service.

The passage then discusses the importance of account security. Users are advised not to share their account information with anyone else and are solely responsible for maintaining the confidentiality and security of their account. Any unauthorized use of the account should be immediately reported to iTunes. To purchase and download iTunes products, users must enter their Apple ID and password or use Touch ID for authentication. Once authenticated, users do not need to re-ent